In [1]:
from keras.models import load_model
import numpy as np
import pandas as pd
import cv2

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 980M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5103)


In [2]:
model = load_model('vgg16_gender2.h5')

In [ ]:
mean = np.load('summ.npy')

In [ ]:

def model_predict(model, img, mean, im_height = 224, im_width = 224, rounded = False):
    image_final = np.zeros((1,224,224,3))
    img = cv2.resize(img, (im_width, im_height))
    img = img - mean
    image_final[0,:,:,:] = img
    image_final = np.swapaxes(image_final, 1, 3)
    image_final = np.swapaxes(image_final, 2, 3)
    
    prediction = model.predict(image_final)
    
    if rounded:
        if prediction > 0.7:
            return "MALE"
        else:
            return "FEMALE"
    
    
    return str(prediction)


def recognize_person():
    gender = ""
    face_cascade = cv2.CascadeClassifier('../Capture-Face-master/face_detection_xml/face_cascade_classifier.xml')
    video_capture = cv2.VideoCapture(0)
    face_detected = False
    while True:
        # Capture frame-by-frame
        ret, frame = video_capture.read()
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
            image = frame[max(0, y - 40): min(frame.shape[0], y+h + 40), max(0, x - 40):min(frame.shape[1], x+w + 40)]
            cv2.imwrite('face.png', image)
            gender = model_predict(model, image, mean)
            face_detected = True
        if face_detected:
            cv2.putText(image, gender, (image.shape[1]/5, image.shape[0]/5), cv2.FONT_HERSHEY_SIMPLEX, 2, 255)
            cv2.imshow("Prediction", image)
            raw_input("Enter To continue")
            face_detected = False
        cv2.imshow('img', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    #When everything is done, release the capture
    video_capture.release()
    
    cv2.destroyAllWindows()

recognize_person()